### Introduction Notes
We are interested in minimizing the error of our function over the test set. The gap between the test set and train set is given approximately as:

$E_{\text{test}} - E_{\text{train}} = k(h/P)^{\alpha}$

If possible we'd like this term to be lower.

P is the number of training examples, so increasing P decreases this gap.
h is a measure of the capacity, so as the capacity goes up this value goes up, which makes sense as it will likely have more capacity to overfit

In practice this is minimized by minimizing $E_{\text{train}} + \beta H (W)$ which is just where we add regularization.

---

We minimize this by computing the gradients and then updating the weights based on these gradients.

Popular procedure is SGD or the "on-line update" where we don't get the complete gradient of the training set, but we get the gradient on the basis of a single or small amount of samples.

### Convolutional Neural Networks
One advantage of convolutional neural networks is that they are better at handling invariance with respect to translations or local distortions. This is because with a large MLP, it could learn the different features in different positions, but it would be very inefficient as it would need multiple little mini networks within the network to handle detecting the important patterns positioned at different locations.

Another issue with fully connected MLPs is that there is kind of this loss of information. The variables can be fed in in any fixed order and the outcome won't change. The information going into an MLP is just a 1d vector of numbers. Yet obviously images have a strong and important 2D structure.

Convolutional networks use "local receptive fields" or patches. They extract the elementary features from them such as edges, corners etc... These features are then combined by subsequent convolutional layers in order to detect higher-order features.

Because feature detectors that are useful on one part of the image are likely to be useful for the entire image, we use the single patch that has one set of weights to go across the entire image.

Units in a layer are organized into planes where all the units share the same set of weights. What this means essentially is that we can kind of think of it as sliding a window of the same weights across the input. This would be opposed to, maybe in the bottom right we have a set of weights that detects a corner here and another set of weights in the top left that detects this corner here.

This does not mean that each individual feature map has the same set of weights across them. That would be quite duplicative for no reason.

In LeNet-5 the first layer has 6 planes or feature maps. A unit in a feature map has 25 inputs connected to a 5x5 area in the input. A sequential implementation (scanning across) would slide this 5x5 set of weights across the input and multiply each by the weight and then add the bias and squash at the end.

Once a feature is detected, the location becomes less important, what's important is the position relative to other features.

To reduce the precision with which the positions are encoded there is sub-sampling of layers which they use averaging but you could also use max pooling or something. This helps to reduce sensitivity to shifts and distortions, also may help with overfitting? Not sure.

Before I go into implementing CNNs I want to rework my code from the previous backprop notebook.

I'm going to, based on Karpathy's micrograd, quickly write up / copy over the code so that I can have an autograd engine that implements .backwards. From there I'll add code on top of this that implements the CNN.

Because I want to practice this, I used o3 to turn the implementation of Value into an assignment with methods that I should implement myself.

In [1]:
#!/usr/bin/env python3
from typing import Iterable, Union
import math
Number = Union[int, float]


class Value:
    """A node in a dynamically‑built computation graph."""

    # -------------------------------------------------------------- T1
    def __init__(self, data: Number, _children: Iterable["Value"] = (), _op: str = ""):
        """Create a **leaf** (when ``_children`` is empty) or an **operation
        result** (when created inside an operator).

        What this method must do
        ------------------------
        1. Store ``data`` – the wrapped scalar (float/int).
        2. Create ``grad`` and set it to **0.0**.  We accumulate partial
           derivatives here during back‑prop.
        3. Track the immediate predecessors with ``self._prev = set(_children)``.
           We use a **set** so look‑ups are O(1) and there are no duplicates.
        4. Save ``_op`` – a short string describing *how* this node was
           produced ("+", "*", "ReLU", etc.).  It is *only* for graphviz /
           debugging convenience.
        5. Initialise ``self._backward`` to a **do‑nothing** lambda.  Each
           operator will later *replace* this with a closure that knows
           how to distribute ``out.grad`` to its parents.

        Why these pieces?
        -----------------
        * Automatic differentiation works by walking the graph from the
          output back to the leaves, so every node needs a list of its
          parents (``_prev``) **and** the rule to push its gradient back
          (``_backward``).
        * ``grad`` starts at 0 so multiple downstream paths can safely
          *accumulate* into it.
        """
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self.prev = set(_children)
        self._op = _op

    def __add__(self, other: Union["Value", Number]):
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        
        return out

    def __mul__(self, other: Union["Value", Number]):
        """self * other."""
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    __rmul__ = __mul__

    def __pow__(self, other: Union["Value", Number]):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data ** other.data, (self, other), '**')
    
        def _backward():
            self.grad += other.data * (self.data ** (other.data - 1)) * out.grad
            self.grad += 0 
            if self.data > 0:
                other.grad += math.log(self.data) * out.data * out.grad
    
        out._backward = _backward
        return out
        
    def relu(self):
        """ReLU activation: max(0, x)."""
        out = Value(max(self.data, 0), (self,), "relu")
        def _backward():
            self.grad += max(0, out.grad)
        out._backward = _backward
        return out

    def backward(self):
        """Compute ``d(output)/d(node)`` for *every* ``node`` that influences
        this Value (call it *out*).

        Behaviour overview
        ------------------
        *The chain rule tells us we must process nodes **in reverse
        topological order** – children **before** parents – so that when
        we reach a node, the gradients flowing into it from all of its
        consumers have already been accumulated.*

        Implementation recipe
        ---------------------
        1. **Topological sort**
           Depth‑first search starting from ``self`` collects nodes in a
           list ``topo`` such that parents appear **before** children.
        2. **Seed the output**
           A node’s gradient with respect to itself is 1, so set
           ``self.grad = 1.0``.
        3. **Reverse sweep**
           Iterate ``for v in reversed(topo): v._backward()``.  Each
           stored ``_backward`` closure adds its *local* contribution to
           ``child.grad``.
        """
        # so we start from my node and go backwards calling backwards on all nodes from
        # root me, all the way to the leaves

        self.grad = 1
        topographical_ancestors = []
        seen = set()
        def recurse(node):
            topographical_ancestors.append(node)
            seen.add(node)
            if(node.prev):
                for node in node.prev:
                    if(node not in seen):
                        recurse(node)
        recurse(self)
        for node in topographical_ancestors:
            node._backward()
            
    def __rpow__(self, base: Union["Value", Number]):
        base = base if isinstance(base, Value) else Value(base)
        out = Value(base.data ** self.data, (base, self), '**')
    
        def _backward():
            base.grad += self.data * (base.data ** (self.data - 1)) * out.grad
            self.grad += math.log(base.data) * out.data * out.grad
    
        out._backward = _backward
        return out

    def log(self):
        out = Value(math.log(self.data), (self,), 'log')
        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other: Union["Value", Number]):
        return self + (-other)

    def __rsub__(self, other: Union["Value", Number]):
        return other + (-self)

    def __truediv__(self, other: Union["Value", Number]):
        return self * other ** -1

    def __rtruediv__(self, other: Union["Value", Number]):
        return other * self ** -1

    def __repr__(self):
        return str(self.data)
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
    
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
    
        return out



In [2]:
## create the neuron, layer and MLP
import random
class Module:
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):
    def __init__(self, nin, nonlinearity):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlinearity = nonlinearity

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        if(self.nonlinearity == "relu"):
            return act.relu()
        return act

    def parameters(self):
        return self.w + [self.b]
    pass
    
class Fully_Connected_Layer(Module):
    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]


class MLP(Module):
    def __init__(self, nin, nouts, nonlinearity):
        sz = [nin] + nouts
        self.layers = [Fully_Connected_Layer(sz[i], sz[i+1], nonlinearity=nonlinearity) for i in range(len(nouts)- 1)]
        self.layers.append(Fully_Connected_Layer(sz[-2], sz[-1], nonlinearity="none"))

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [38]:
## test micrograd on simple excercise from past paper like the adder problem
data_adder = [
    ([0,0,0,0], [0,0,0]), ([0,0,0,1], [0,0,1]), ([0,0,1,0], [0,1,0]), ([0,0,1,1], [0,1,1]),
    ([0,1,0,0], [0,0,1]), ([0,1,0,1], [0,1,0]), ([0,1,1,0], [0,1,1]), ([0,1,1,1], [1,0,0]),
    ([1,0,0,0], [0,1,0]), ([1,0,0,1], [0,1,1]), ([1,0,1,0], [1,0,0]), ([1,0,1,1], [1,0,1]),
    ([1,1,0,0], [0,1,1]), ([1,1,0,1], [1,0,0]), ([1,1,1,0], [1,0,1]), ([1,1,1,1], [1,1,0])
]

adder_mlp = MLP(4, [16, 16, 3], "relu")
learning_rate = 0.02
epochs = 200
print("running")
for epoch in range(epochs):
    average_loss_cum = Value(0)
    for x,y in data_adder:
        adder_mlp.zero_grad()
        out=adder_mlp(x)
        loss = Value(0)
        for i in range(len(y)):
            loss += (1/2) * ((Value(y[i]) - out[i]) ** 2)
        loss.backward()
        average_loss_cum += loss
        for param in adder_mlp.parameters():
            param.data -= learning_rate * param.grad
    if(epoch % 50 == 0):
        print(average_loss_cum / len(data_adder))

# evaluation from last time
print(f"\n--- Evaluation on Adder Problem ---")
total_samples = len(data_adder)
correct_predictions = 0

for x_sample, y_true_list in data_adder:
    out = adder_mlp(x_sample)
    predicted_bits = [1 if o.data > 0.5 else 0 for o in out]
    is_sample_correct = (predicted_bits == y_true_list)
    
    if is_sample_correct:
        correct_predictions += 1
        
    status_color = "\033[92mCORRECT\033[0m" if is_sample_correct else "\033[91mWRONG\033[0m"
    
    print(f"Input: {x_sample}, Expected: {y_true_list}, Got_Bits: {predicted_bits} --> {status_color}")

accuracy = (correct_predictions / total_samples) * 100
print(f"\nOverall Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_samples})")

running
2.2032093392464085
0.21252774821545908
0.15229204764947635
0.12024741910287676

--- Evaluation on Adder Problem ---
Input: [0, 0, 0, 0], Expected: [0, 0, 0], Got_Bits: [0, 1, 0] --> WRONG
Input: [0, 0, 0, 1], Expected: [0, 0, 1], Got_Bits: [0, 1, 1] --> WRONG
Input: [0, 0, 1, 0], Expected: [0, 1, 0], Got_Bits: [0, 1, 0] --> CORRECT
Input: [0, 0, 1, 1], Expected: [0, 1, 1], Got_Bits: [0, 1, 1] --> CORRECT
Input: [0, 1, 0, 0], Expected: [0, 0, 1], Got_Bits: [0, 0, 1] --> CORRECT
Input: [0, 1, 0, 1], Expected: [0, 1, 0], Got_Bits: [1, 0, 0] --> WRONG
Input: [0, 1, 1, 0], Expected: [0, 1, 1], Got_Bits: [0, 1, 1] --> CORRECT
Input: [0, 1, 1, 1], Expected: [1, 0, 0], Got_Bits: [1, 0, 0] --> CORRECT
Input: [1, 0, 0, 0], Expected: [0, 1, 0], Got_Bits: [0, 1, 0] --> CORRECT
Input: [1, 0, 0, 1], Expected: [0, 1, 1], Got_Bits: [0, 1, 1] --> CORRECT
Input: [1, 0, 1, 0], Expected: [1, 0, 0], Got_Bits: [1, 0, 0] --> CORRECT
Input: [1, 0, 1, 1], Expected: [1, 0, 1], Got_Bits: [1, 0, 1] --> CO

In [3]:
## bring in MNIST and setup some convenience functions
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Optionally normalize (pixel values between 0 and 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert to plain lists (if you don't want NumPy arrays)
x_train_list = x_train.tolist()
y_train_list = y_train.tolist()
print("done")

done


In [38]:
# let's work with a smaller set
# 28 x 28 examples
examples = 100
x = x_train_list[:examples]
y = y_train_list[:examples]

In [13]:
# helper function to visualize an example
import matplotlib.pyplot as plt
import numpy as np

def show_mnist_example(image, pred_label=None, true_label=None):
    img = np.array(image)
    # Set up label text
    title = ""
    if true_label is not None:
        title += f"True: {true_label}  "
    if pred_label is not None:
        title += f"Pred: {pred_label}"

    plt.imshow(img, cmap='gray')
    plt.axis('off')
    if title:
        plt.title(title, fontsize=12)
    plt.show()

In [39]:
x_flat = []
for i in x:
    acc = []
    for j in i:
        acc += j
    x_flat.append(acc)

In [21]:
e = 2.71828

def loss_individual(model_outputs, target_label_index):
    max_output = max(model_outputs, key=lambda v: v.data if hasattr(v, 'data') else float(v))
    shifted_outputs = [val - max_output for val in model_outputs]

    exps = [Value(math.e) ** val for val in shifted_outputs]
    sum_exps = Value(0)
    for val in exps:
        sum_exps += val

    target_exp = exps[target_label_index]
    normalized_target = target_exp / sum_exps

    loss = -normalized_target.log()
    return loss

In [22]:
## Test running just a fully connected network on mnist with yann le cunns as well as cross
## test micrograd on simple excercise from past paper like the adder problem
lossi = []
mnist_mlp = MLP(28*28, [64, 64, 10], "relu")
learning_rate = 0.005 #3e-4 per karpathy's love lol
epochs = 5
print("running")
print(len(mnist_mlp.parameters()))
for epoch in range(epochs):
    average_loss_cum = Value(0)
    for i in range(len(x_flat)):
        mnist_mlp.zero_grad()
        out=mnist_mlp(x_flat[i])
        loss = loss_individual(out, y[i])
        loss.backward()
        average_loss_cum += loss
        for param in mnist_mlp.parameters():
            param.data -= learning_rate * param.grad
    print(average_loss_cum / len(x_flat))
    lossi.append(average_loss_cum / len(x_flat))
print("done")

running
55050
90.19601231180822
57.80504646856195
45.80947132662187
65.92117384111525
49.03913192907086
done


In [35]:
# test it on 10 examples
test_number = 20
# helper function to visualize an example
import matplotlib.pyplot as plt
import numpy as np

def pick_label(out):
    largest = 0
    index = 0
    for i in range(len(out)):
        if(out[i].data > largest):
            largest = out[i].data
            index = i
    return index
        
for i in range(test_number):
    forward_label = mnist_mlp(x_flat[i])
    show_mnist_example(x[i], pred_label=pick_label(forward_label), true_label=y[i])
    

In [40]:
# get correct percentage on all examples
correct = 0

for i in range(len(x_flat)):
    forward_label = mnist_mlp(x_flat[i])
    pred = pick_label(forward_label)
    true = y[i]

    if(i % 10 == 0):
        print("calculating...")
    
    if pred == true:
        correct += 1

accuracy = (correct / len(x_flat) * 100
print(f"Accuracy: {accuracy:.2f}%")


calculating...
calculating...
calculating...
calculating...
calculating...
calculating...
calculating...
calculating...
calculating...
calculating...
Accuracy: 180.00%


In [42]:
accuracy = (correct / len(x_flat)) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 36.00%


okay this takes years and isn't even very good lol, but I guess for just running a random MLP for 5 minutes, not tuning any hyper parameters, making it extremely small, it's not horrible. Show's the power of MLPs! 36% accuracy when a random guesser would be 10% right

I guess we're simulating the tech from that era in terms of cpu and how slow everything is lmao

So what does a single convolutional cell do? For simplicity I'm going to ignore stride and padding here for now.

We have a filter, which in this case will be 2D. The filter will have a height and a width. It will also have a bias term.

The filter is then slide across the input tensor. So it will slide over (input_width - filter_width) * (input_height - filter_height) times. At each of these positions / slides it will compute the dot product between the filter and the patch of the input. To compute the dot product we multiply each filter position with the corresponding input position. Then we sum all of those multiplications up. After calculating the dot product we add the bias. Then we apply our non-linearity (relu) to it and put it in the output.



In [ ]:
# now let's implement the convolutional layer and max /average pooling layer and see how those do
# if this becomes too slow, and I realize now that I may need to do a torch implementation because this autograd engine
# is very slow lol. I guess because I'm using python lists and non-vectorized code everywhere

class Convolutional_Unit(Module):
    def __init__(self, kernel_height, kernel_width, **kwargs):
        self.kernel_width = kernel_width
        self.kernel_height = kernel_height
        self.kernel = []
        for i in range(kernel_height):
            self.kernel.append([Value(random.uniform(-1,1)) for _ in range(kernel_width)])
        self.b = Value(0)
    
    def __call__(self, x):
        out_height = len(x) - self.kernel_height
        out_width = len(x[0]) - self.kernel_width
        # slide it over the input, return a 2d list of values
        out = []
        for i in range(out_height):
            row = []
            for j in range(out_width):
                # quadrouple nested for loop lol
                # there's definitely a lot ofrepeated work here, maybe I should make a map of values later and store them lol
                # or just vectorize the code lol
                acc = Value(0)
                for kernel_i in range(kernel_height):
                    for kernel_j in range(kernel_width):
                        acc += self.kernel[kernel_i][kernel_j] * x[i + kernel_i][j + kernel_j]
                acc += self.b
                row.append(acc)
            out.append(row)
        return out

    def parameters(self):
        return self.kernel + [self.b]

class Convolutional_Feature_Map(Module):
    def __init__(self, num_units, kernel_height, kernel_width, **kwargs):
        self.convolutional_units = [Convolutional_Unit(kernel_height, kernel_width) for _ in range(num_units)]

    def __call__(self, x):
        out = [c(x) for c in self.convolutional_units]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for c in self.convolutional_units for p in c.parameters()]
    
class Average_Pooling_Layer(Module):
    def __init__(self, window_height, window_width, stride_height, stride_width, nonlinearity):
        self.window_width = kernel_width
        self.window_height = kernel_height
        self.stride_height = stride_height
        self.stride_width = stride_width

    def __call__(self, x):
        # for simplicity we have to ensure that the input size % stride == 0
        if(len(x) % self.stride_height != 0 or len(x[0]) % self.stride_width != 0:
            print("need to make it divisible bruh")
            return
        out_height = len(x) // self.stride_height
        out_width = len(x[0]) // self.stride_width
        for i in range(out_height):
            for j in range(out_width):
                # HEMMINGWAY BRIDGE
                # FINISH IMPLEMENTING THE AVERAGE POOLING HERE
                
        
        
# class Max_Pooling_Layer(Module):

        
# we'll also need some sort of class that can wrap things and chain them together


In [ ]:
# test out how our new convolutional neural net learns

In [ ]:
# make better error metrics / reporting and make it more professional, train test set, charts, validations

In [ ]:
# do a bit of hyper parameter tuning

In [ ]:
## cap it here, go through appendecies to see if there's other things to understand
# test the pre-condition for faster convergance and scale the tanh
# do stochastic vs batch gradient descent
# try their stochastic diagonal levenberg-marquardt / understand that